<a href="https://colab.research.google.com/github/f-player/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1) Установка зависимостей
!pip -q install -U transformers accelerate torch datasets evaluate sacrebleu rouge-score sentence-transformers faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 73.3 MB/s eta 0:00:00


In [2]:
# 2) Импорт
import re, pandas as pd, numpy as np
from tqdm.auto import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
from sentence_transformers import SentenceTransformer, util

# Метрики
rouge = evaluate.load("rouge")
bleu = evaluate.load("sacrebleu")

# Нормализация, EM, F1
def normalize_text(s: str) -> str:
    if s is None: return ""
    import re
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s\-–—.,?!:;()«»\"'`]", "", s, flags=re.UNICODE)
    return s

def exact_match(pred, gold):
    return 1.0 if normalize_text(pred) == normalize_text(gold) else 0.0

def f1_token(pred, gold):
    p_tokens = normalize_text(pred).split()
    g_tokens = normalize_text(gold).split()
    if len(p_tokens) == 0 and len(g_tokens) == 0:
        return 1.0
    if len(p_tokens) == 0 or len(g_tokens) == 0:
        return 0.0
    common = {}
    for t in g_tokens:
        common[t] = common.get(t,0)+1
    num_same = 0
    for t in p_tokens:
        if common.get(t,0) > 0:
            num_same += 1
            common[t] -= 1
    if num_same == 0:
        return 0.0
    precision = num_same / len(p_tokens)
    recall    = num_same / len(g_tokens)
    return 2 * precision * recall / (precision + recall)

# Семантическая близость
sem_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
def cosine_sim(a, b):
    emb = sem_model.encode([a, b], convert_to_tensor=True, normalize_embeddings=True)
    return float(util.cos_sim(emb[0], emb[1]).item())


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
from google.colab import files
uploaded = files.upload()
csv_path = list(uploaded.keys())[0]

df = pd.read_csv(csv_path)
cols_lower = {c.lower(): c for c in df.columns}
cands_q = ["question","q","prompt","query","question_text","вопрос"]
cands_a = ["answer","a","completion","response","target","gold","ответ"]

QUESTION_COL = next((cols_lower[c] for c in cands_q if c in cols_lower), None)
ANSWER_COL   = next((cols_lower[c] for c in cands_a if c in cols_lower), None)

if QUESTION_COL is None or ANSWER_COL is None:
    raise ValueError("Не найдены колонки вопроса/ответа. Укажите вручную.")

df = df[[QUESTION_COL, ANSWER_COL]].rename(columns={QUESTION_COL:"question", ANSWER_COL:"answer"}).dropna().reset_index(drop=True)
print("Строк:", len(df))
df.head(3)


Saving qa_only.csv to qa_only.csv
Строк: 1000


,question,answer
0,How does Freud's psychoanalytic theory explain...,Freud believed that human behaviour is driven ...
1,How does behaviourism differ from psychoanalyt...,behaviourism suggests that behaviour is learne...
2,What is the main criticism of Carl Rogers' per...,Critics argue that it places too much emphasis...


In [4]:
# Эмбеддинг базы
corpus = df["answer"].tolist()
corpus_emb = sem_model.encode(corpus, convert_to_tensor=True, normalize_embeddings=True)

def retrieve_context(query, topk=3):
    q_emb = sem_model.encode(query, convert_to_tensor=True, normalize_embeddings=True)
    scores = util.cos_sim(q_emb, corpus_emb)[0]
    top_idx = torch.topk(scores, k=topk).indices.cpu().tolist()
    return [corpus[i] for i in top_idx]


In [5]:
MODEL_NAME = "google/flan-t5-base"  # можно: "google/mt5-base"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [7]:
def build_prompt(q, ctxs):
    context = "\n".join([f"- {c}" for c in ctxs])
    return f"""Use the following context to answer the question.

Context:
{context}

Question: {q}
Answer:"""

GEN_KW = dict(
    max_new_tokens=64,
    do_sample=False,
    temperature=0.0,
    num_beams=1
)

preds = []
for q in tqdm(df["question"].tolist(), desc="Generating with RAG"):
    ctxs = retrieve_context(q, topk=3)
    prompt = build_prompt(q, ctxs)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    with torch.no_grad():
        out = model.generate(**inputs, **GEN_KW)
    text = tokenizer.decode(out[0], skip_special_tokens=True).strip()
    preds.append(text)

df["pred"] = preds
df.head(5)


Generating with RAG:   0%|          | 0/1000 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignore

,question,answer,pred
0,How does Freud's psychoanalytic theory explain...,Freud believed that human behaviour is driven ...,unconscious thoughts and emotions influence be...
1,How does behaviourism differ from psychoanalyt...,behaviourism suggests that behaviour is learne...,The primary focus of behaviourism is on observ...
2,What is the main criticism of Carl Rogers' per...,Critics argue that it places too much emphasis...,it emphasises the importance of creating a saf...
3,What is the main focus of cognitive psychology?,The main focus of cognitive psychology is on h...,how mental processes work
4,What is the main difference between classical ...,Classical conditioning involves associating a ...,classical conditioning involves learning throu...


In [8]:
# Построчные
df["em"]  = [exact_match(p, g) for p, g in zip(df["pred"], df["answer"])]
df["f1"]  = [f1_token(p, g) for p, g in zip(df["pred"], df["answer"])]
df["cos"] = [cosine_sim(p, g) for p, g in zip(df["pred"], df["answer"])]

# Коллективные
rouge_res = rouge.compute(predictions=df["pred"].tolist(),
                          references=df["answer"].tolist(), use_stemmer=True)
bleu_res  = bleu.compute(predictions=df["pred"].tolist(),
                         references=[[a] for a in df["answer"].tolist()])

summary = {
    "EM_mean": float(np.mean(df["em"])),
    "F1_mean": float(np.mean(df["f1"])),
    "Cosine_mean": float(np.mean(df["cos"])),
    "ROUGE1": rouge_res.get("rouge1", 0.0),
    "ROUGE2": rouge_res.get("rouge2", 0.0),
    "ROUGEL": rouge_res.get("rougeL", 0.0),
    "ROUGELsum": rouge_res.get("rougeLsum", 0.0),
    "BLEU": bleu_res.get("score", 0.0)
}

# 🔹 Интегральная метрика
def aggregate_score(summary,
                    w_em=0.1, w_f1=0.2, w_rouge=0.2, w_bleu=0.2, w_cos=0.3):
    bleu_norm = summary["BLEU"] / 100 if summary["BLEU"] > 1 else summary["BLEU"]
    rouge_val = summary.get("ROUGEL", summary.get("ROUGELsum", 0.0))
    score = (
        w_em * summary["EM_mean"] +
        w_f1 * summary["F1_mean"] +
        w_rouge * rouge_val +
        w_bleu * bleu_norm +
        w_cos * summary["Cosine_mean"]
    )
    return score

final_score = aggregate_score(summary)
print("Интегральная метрика RAG:", round(final_score, 3))
summary


Интегральная метрика RAG: 0.448


{'EM_mean': 0.045,
 'F1_mean': 0.44744884318962136,
 'Cosine_mean': 0.6811499944664537,
 'ROUGE1': np.float64(0.48992595979199755),
 'ROUGE2': np.float64(0.398450209036763),
 'ROUGEL': np.float64(0.47199559635776117),
 'ROUGELsum': np.float64(0.4727249721257125),
 'BLEU': 27.854821339418027}

In [9]:
out_csv = "qa_predictions_with_metrics_RAG.csv"
df.to_csv(out_csv, index=False)
print("Сохранено:", out_csv)
df.head(10)


Сохранено: qa_predictions_with_metrics_RAG.csv


,question,answer,pred,em,f1,cos
0,How does Freud's psychoanalytic theory explain...,Freud believed that human behaviour is driven ...,unconscious thoughts and emotions influence be...,0.0,0.242424,0.654939
1,How does behaviourism differ from psychoanalyt...,behaviourism suggests that behaviour is learne...,The primary focus of behaviourism is on observ...,0.0,0.270270,0.754017
2,What is the main criticism of Carl Rogers' per...,Critics argue that it places too much emphasis...,it emphasises the importance of creating a saf...,0.0,0.139535,0.307436
3,What is the main focus of cognitive psychology?,The main focus of cognitive psychology is on h...,how mental processes work,0.0,0.272727,0.777791
4,What is the main difference between classical ...,Classical conditioning involves associating a ...,classical conditioning involves learning throu...,0.0,0.285714,0.858320
5,What is the primary focus of behaviourism?,The primary focus of behaviourism is on observ...,observable behaviour,0.0,0.105263,0.593914
6,What is the primary focus of humanistic psycho...,The primary focus of humanistic psychology is ...,the study of the whole person,0.0,0.166667,0.455403
7,How does Maslow's hierarchy of needs relate to...,Maslow's hierarchy of needs suggests that peop...,proposes that basic physiological needs must b...,0.0,0.250000,0.698137
8,What is the main criticism of the biological a...,Critics argue that it reduces complex human be...,it uses evolutionary explanations to explain b...,0.0,0.181818,0.485532
9,What is the main difference between nature and...,Nature refers to genetic or innate factors tha...,Nature refers to the genetic or biological fac...,0.0,0.711111,0.946611
